In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math

In [ ]:
df_treino = pd.read_csv("data/train.csv")
df_treino.head()

In [ ]:
df_treino.info()

In [ ]:
df_treino.nunique()

In [ ]:
categorical_columns = df_treino.select_dtypes(include=['object'])
categorical_columns.columns

## Adicionar cálculo do IMC ao dataframe

In [ ]:
df_treino['IMC'] = df_treino['Weight']/(pow(df_treino['Height'],2))

In [ ]:
def resumir_df(df):
    print(f'Formato do DataFrame:{df.shape}')
    df_resumo = pd.DataFrame(df.dtypes, columns=['Tipo'])
    df_resumo['Valores faltantes'] = df.isnull().sum().values
    df_resumo['Percentual de valores faltantes'] = (df.isnull().sum().values / len(df) * 100)
    df_resumo['Valores únicos'] = df.nunique().values
    desc = pd.DataFrame(df.describe(include='all').transpose())
    df_resumo['Min'] = desc['min'].values
    df_resumo['Máx'] = desc['max'].values
    return df_resumo.sort_values(by='Valores únicos', ascending=True)
resumir_df(df_treino)

## Significado das siglas das variáveis
- FAVC: Frequência de consumo de comidas altamente calóricas
- CAEC: Frequência de pequenas refeições entre as refeições principais
- SMOKE: Variável binária que indica fumante ou não fumante
- SCC: Variável binária que indica monitoramento de calorias consumidas
- CALC: Variável binária que indica o consumo de álcool
- MTRANS: Meio de transporte mais utilizado
- NCP: Número de refeições por dia
- FCVC: Frequência de consumo de vegetais em refeições
- TUE: Tempo gasto em dispositivos tecnológicos 
- FAF: Frequência de atividade física
- CH2O: Quantidade (em litros) de consumo diário de água

## Análise de frequência das variáveis

In [ ]:
def plot_variables_frequency(df: pd.DataFrame):

    cols_to_plot = [col for col in df.columns if col not in ('id')]
    num_variables = len(df.columns)
    num_columns = 3
    num_rows = math.ceil(num_variables / num_columns)
    
    plt.figure(figsize=(15, 5 * num_rows))  # Adjust the figure size accordingly

    
    for i, col in enumerate(cols_to_plot):
        plt.subplot(num_rows, num_columns, i % num_columns + 1 + (i // num_columns) * num_columns)
        if df[col].dtype == 'object':
            sns.histplot(df, y=col, bins=10, color='green', legend=False, shrink=.8)
        else:
            sns.histplot(df, x=col, bins=10, color='green', kde=True)
        plt.title(f'Histograma de {col}')
        

    plt.tight_layout()
    plt.show()
plot_variables_frequency(df_treino)

## Análise multivariável

In [ ]:
def plot_variables_boxplot(df: pd.DataFrame):

    cols_to_plot = [col for col in df.columns if (col not in ('id', 'NObeyesdad') and df[col].dtype!='object')]
    num_variables = len(df.columns)
    num_columns = 2
    num_rows = math.ceil(num_variables / num_columns)
    
    plt.figure(figsize=(15, 5 * num_rows))  # Adjust the figure size accordingly
    obesity_level_order = ['Obesity_Type_III', 'Obesity_Type_II', 'Obesity_Type_I', 'Overweight_Level_II', 
                           'Overweight_Level_I', 'Normal_Weight', 'Insufficient_Weight']
    
    for i, col in enumerate(cols_to_plot):
        plt.subplot(num_rows, num_columns, i % num_columns + 1 + (i // num_columns) * num_columns)
        sns.boxplot(df, y='NObeyesdad', x=col, hue='NObeyesdad', order=obesity_level_order)
        plt.title(f'Boxplot de {col}')
        

    plt.tight_layout()
    plt.show()
plot_variables_boxplot(df_treino)

In [ ]:
def plot_variables_grouped_by_obesity(df: pd.DataFrame):

    cols_to_plot = [col for col in df.columns if (col not in ('id', 'NObeyesdad') and df[col].dtype=='object')]
    num_variables = len(cols_to_plot)
    num_columns = 2
    num_rows = math.ceil(num_variables / num_columns)
    
    fig, axs = plt.subplots(num_rows, num_columns, figsize=(15, num_rows * 5))
    obesity_level_order = ['Insufficient_Weight', 'Normal_Weight', 'Overweight_Level_I', 'Overweight_Level_II', 
                           'Obesity_Type_I', 'Obesity_Type_II', 'Obesity_Type_III']
    
    for i, col in enumerate(cols_to_plot):
        col_data = df.groupby('NObeyesdad')[col].value_counts().unstack()
        col_data = col_data.reindex(obesity_level_order)
        col_data.plot(kind='bar', stacked=True, ax=axs[i//num_columns, i%num_columns])
        axs[i//num_columns, i%num_columns].set_title(f'Agrupamento de {col}')
        axs[i//num_columns, i%num_columns].tick_params(axis='x', rotation=30)
        

    plt.tight_layout()
    plt.show()
plot_variables_grouped_by_obesity(df_treino)